# Northeastern Illinois University (NEIU) Current Conditions

In [1]:
from IPython.display import IFrame
import pandas as pd
import sage_data_client
import holoviews as hv
import hvplot.pandas
from metpy.units import units
import metpy.calc as mpcalc
from bokeh.models import DatetimeTickFormatter
import sage_data_client

hv.extension("bokeh")

def apply_formatter(plot, element):
    plot.handles['xaxis'].formatter = DatetimeTickFormatter(hours='%m/%d/%Y \n %l:%M %p',
                                                            minutes='%m/%d/%Y \n %l:%M %p',
                                                            hourmin='%m/%d/%Y \n %l:%M %p',
                                                            days='%m/%d/%Y \n %l:%M %p',
                                                            months='%m/%d/%Y \n %l:%M %p')

def add_local_time(df):
    df["Local_Time"] = pd.to_datetime(df.index,unit='s', utc=True)\
                          .map(lambda x: x.tz_convert('America/Chicago'))
    return df

def get_field_convert_to_degF(df, variable):
    subset = df.loc[df.name == variable]

    subset["value"] = (subset.value.values * units.degC).to("degF")
    return subset.value

def get_field(df, variable):
    subset = df.loc[df.name == variable]
    return subset.value

import warnings
warnings.filterwarnings('ignore')

## Temperature and Dewpoint

In [2]:
wxt_df = sage_data_client.query(
    start="-12h", 
    filter={
        "vsn": "W08D",
        "name": "wxt.env.temp|wxt.env.humidity"
    }
).set_index("timestamp")

temp_df = wxt_df.loc[wxt_df.name == "wxt.env.temp"]
hum_df = wxt_df.loc[wxt_df.name == "wxt.env.humidity"]

min_df = temp_df[["value"]].resample("1min").mean()
min_df["rh"] = hum_df[["value"]].resample("1min").mean()

min_df["tmpf"] = (min_df.value.values * units.degC).to("degF").m
min_df["dwpf"] = mpcalc.dewpoint_from_relative_humidity(min_df.tmpf.values * units.degF,
                                       min_df.rh.values * units.percent).to("degF").m

min_df = add_local_time(min_df)

temp_plot = min_df.hvplot(x='Local_Time',
                          y='tmpf',
                          color = "r",
                          label="Temperature [degF]"
                         )
dewp_plot = min_df.hvplot(x='Local_Time',
                          y='dwpf',
                          color="g",
                          title="Current Weather at Northeastern Illinois University (NEIU)",
                          label="Dewpoint [degF]",
                          ylabel="Temperature | Dewpoint \n [degF]",
                          xlabel="Local Time (Central/US)")

(temp_plot * dewp_plot).opts(hooks=[apply_formatter], legend_position='right')

:Overlay
   .Curve.Temperature_left_square_bracket_degF_right_square_bracket :Curve   [Local_Time]   (tmpf)
   .Curve.Dewpoint_left_square_bracket_degF_right_square_bracket    :Curve   [Local_Time]   (dwpf)

## Air Quality

In [3]:
df = sage_data_client.query(
    start="-12h", 
    filter={
        "vsn": "W08D",
        "name": "aqt.particle.pm2.5"
    }
).set_index("timestamp")

df = add_local_time(df)[["value", "Local_Time"]].resample("5min").mean()

df.hvplot(label='PM 2.5 Values at Northeastern Illinois University (NEIU)',
          x='Local_Time',
          y='value',
          ylabel='PM 2.5 \n [microgram per cubic meter]',
          xlabel="Local Time (Central/US)").opts(hooks=[apply_formatter])

:Curve   [Local_Time]   (value)

## Soil Temperature

In [11]:
df = sage_data_client.query(
    start="-1d", 
    filter={
        "vsn": "W08D",
        "name": "temp_d1|temp_d2|temp_d3|temp_d4",
        "deviceName": "MFR node MNLA4O107"
    }
).set_index("timestamp")

soil_temp_df = pd.DataFrame({"~6 inches below the surface": get_field_convert_to_degF(df, "temp_d1"),
                             "~1 feet below the surface": get_field_convert_to_degF(df, "temp_d2"),
                             "~1.5 feet below the surface": get_field_convert_to_degF(df, "temp_d3"),
                             "~2 feet below the surface": get_field_convert_to_degF(df, "temp_d4")})

soil_temp_df = add_local_time(soil_temp_df)

first = soil_temp_df.hvplot(ylabel="Soil Temperature [degF]",
                    xlabel="Time/Date",
                    x='Local_Time',
                    title="Soil Temperature at Northeastern Illinois University (NEIU) \n North Field Site").opts(hooks=[apply_formatter])

df = sage_data_client.query(
    start="-1d", 
    filter={
        "vsn": "W08D",
        "name": "temp_d1|temp_d2|temp_d3|temp_d4",
        "deviceName": "MFR node MNLA4O108"
    }
).set_index("timestamp")

soil_temp_df = pd.DataFrame({"~6 inches below the surface": get_field_convert_to_degF(df, "temp_d1"),
                             "~1 feet below the surface": get_field_convert_to_degF(df, "temp_d2"),
                             "~1.5 feet below the surface": get_field_convert_to_degF(df, "temp_d3"),
                             "~2 feet below the surface": get_field_convert_to_degF(df, "temp_d4")})

soil_temp_df = add_local_time(soil_temp_df)

second = soil_temp_df.hvplot(ylabel="Soil Temperature [degF]",
                    xlabel="Time/Date",
                    x='Local_Time',
                    title="Soil Temperature at Northeastern Illinois University (NEIU) \n South Field Site").opts(hooks=[apply_formatter])

(first + second).cols(1)

:Layout
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [Local_Time]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Curve   [Local_Time]   (value)

## Soil Moisture

In [12]:
df = sage_data_client.query(
    start="-1d", 
    filter={
        "vsn": "W08D",
        "name": "vwc_d1|vwc_d2|vwc_d3|vwc_d4",
        "deviceName": "MFR node MNLA4O107"
    }
).set_index("timestamp")

soil_temp_df = pd.DataFrame({"~6 inches below the surface": get_field(df, "vwc_d1"),
                             "~1 feet below the surface": get_field(df, "vwc_d2"),
                             "~1.5 feet below the surface": get_field(df, "vwc_d3"),
                             "~2 feet below the surface": get_field(df, "vwc_d4")})

soil_temp_df = add_local_time(soil_temp_df)

first = soil_temp_df.hvplot(ylabel="Soil Moisture [%]",
                    xlabel="Time/Date",
                    x='Local_Time',
                    title="Soil Moisture at Northeastern Illinois University (NEIU) \n North Field Site").opts(hooks=[apply_formatter])

df = sage_data_client.query(
    start="-1d", 
    filter={
        "vsn": "W08D",
        "name": "vwc_d1|vwc_d2|vwc_d3|vwc_d4",
        "deviceName": "MFR node MNLA4O108"
    }
).set_index("timestamp")

soil_temp_df = pd.DataFrame({"~6 inches below the surface": get_field(df, "vwc_d1"),
                             "~1 feet below the surface": get_field(df, "vwc_d2"),
                             "~1.5 feet below the surface": get_field(df, "vwc_d3"),
                             "~2 feet below the surface": get_field(df, "vwc_d4")})

soil_temp_df = add_local_time(soil_temp_df)

second = soil_temp_df.hvplot(ylabel="Soil Moisture [%]",
                    xlabel="Time/Date",
                    x='Local_Time',
                    title="Soil Moisture at Northeastern Illinois University (NEIU) \n South Field Site").opts(hooks=[apply_formatter])

(first + second).cols(1)

:Layout
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [Local_Time]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Curve   [Local_Time]   (value)

## Location
Observations the Northeastern Illinois University (NEIU) campus, near the science building.

<div>
<img src="../../images/neiu-rooftop.JPG" width="500"/>
</div>

### Rooftop Location (Weather + Air Quality)

In [8]:
IFrame("https://www.google.com/maps/embed?pb=!1m17!1m12!1m3!1d2958.1959025119095!2d-87.71920398835765!3d41.98053297111069!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!3m2!1m1!2zNDHCsDU4JzQ5LjkiTiA4N8KwNDInNTkuOSJX!5e1!3m2!1sen!2sus!4v1745333727055!5m2!1sen!2sus", width=700, height=350)

### North Field Site

In [9]:
IFrame("https://www.google.com/maps/embed?pb=!1m17!1m12!1m3!1d286.2041185919134!2d-87.71759675966793!3d41.981453922460354!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!3m2!1m1!2zNDHCsDU4JzUzLjMiTiA4N8KwNDMnMDIuMyJX!5e1!3m2!1sen!2sus!4v1745333809016!5m2!1sen!2sus", width=700, height=350)

### South Field Site

In [10]:
IFrame("https://www.google.com/maps/embed?pb=!1m17!1m12!1m3!1d562.2794205611194!2d-87.71699885841392!3d41.97745286791864!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!3m2!1m1!2zNDHCsDU4JzM5LjAiTiA4N8KwNDInNTkuMyJX!5e1!3m2!1sen!2sus!4v1745333857044!5m2!1sen!2sus", width=700, height=350)